In [1]:
import pickle
import numpy as np
import pandas as pd
import cv2
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import glob
import os
import pdb
#import ipynb.fs.full.AnglesFuncs as af

In [2]:
#--------------------------------------------------------------
def FindThreshold(FFT, alpha):
    TH = np.mean(FFT)
    ref = 0.1
    totAmp = np.sum(FFT)
    diff = 100
    i = 0
    
    while ((np.abs(diff) > ref) | (i < 100)):
        _, thresh = cv2.threshold(FFT, TH, 1,
                                  cv2.THRESH_BINARY)
        relAmp = np.sum(FFT*thresh)
        # number of bins over the TH
        n = np.sum(thresh)
        diff = (alpha*totAmp - relAmp) / n
        
        TH += (-1 * diff /50)
        i += 1
        
    return TH

#--------------------------------------------------------------
def draw_vector(img, v0, v1):
    cv2.line(img, v0, v1, (0,255,0),10)
    #ax = ax or plt.gca()
    #arrowprops=dict(arrowstyle='->',linewidth=2,shrinkA=0, shrinkB=0)
    #ax.annotate('', v1, v0, arrowprops=arrowprops)

#--------------------------------------------------------------
def find_main_component_dft(item, th):
    # item: address of the image
    # th: Threshold
    # figAddress: address of the figure folder
    figfolder = "figures/"
    
    # extracting info from the name
    idx = [i for i in range(len(item)) if item[i] == '/']
    folder = item[idx[1]:np.max(idx)+1]
    name = item[np.max(idx)+1:-4]
    
    image = cv2.imread(item)
    
    # dft and mag
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dftIm = cv2.dft(np.float32(image),flags = cv2.DFT_COMPLEX_OUTPUT)
    dftImShift = np.fft.fftshift(dftIm)
    magIm = 20*np.log(cv2.magnitude(dftImShift[:,:,0],dftImShift[:,:,1]))
    
    size_img = image.shape
    
    # plot gray image
    plt.figure(figsize=(20,12))
    plt.imshow(image, cmap = 'gray')
    plt.title('Original Image', fontsize=25)
    plt.xlabel('x', fontsize=20)
    plt.ylabel('y', fontsize=20)
    plt.xticks()
    plt.yticks()
    plt.savefig(folder + figfolder + name + '_gry.png')
    
    # plot magnitude
    plt.figure(figsize=(20,12.3))
    plt.imshow(magIm, cmap = 'gray')
    plt.axis('equal')
    plt.title('Magnitude of FT', fontsize=25)
    plt.xlabel('$\omega_x$', fontsize=20)
    plt.ylabel('$\omega_y$', fontsize=20)
    plt.colorbar()
    plt.savefig(folder + figfolder + name + '_mag.png')
    
    # Threshold
    #selection of magnitudes for biary picture: if the element in magIm >= maxMag/5*th => element = maxMag,
    # else is 0.
    maxMag = np.max(magIm)
    TH = FindThreshold(magIm, 1/20)
    ret, thresh = cv2.threshold(magIm, TH, maxMag, \
                                cv2.THRESH_BINARY)
    
    # Principal Component Analysis
    y,x = np.where(thresh == np.max(thresh))
    y = -y
    X = np.ones((len(x),2))
    X[:,0] = x; X[:,1] = y
    pca = PCA(n_components=2)
    pca.fit(X)
    
    # converting from gray to color (because we need it in black
    # and white. very tricky)
    thresh = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
    
    # drawing vectors of components on threshold
    for sigma, direction in zip(pca.explained_variance_, pca.components_):
        v = direction.T * 2 * np.sqrt(sigma)
        zero = [np.mean(x), np.mean(y)]
        #draw_vector(thresh, zero, zero - v)
        v0 = np.abs(zero)
        v0 = v0.astype(int)
        v1 = np.abs(zero-v)
        v1 = v1.astype(int)
        cv2.line(thresh, (v0[0], v0[1]), (v1[0], v1[1]), (255,0,0),10)
        
    plt.axis('equal');
    
    # Plot Threshold
    plt.figure(figsize=(20,15.4))
    plt.imshow(thresh)
    plt.axis('equal')
    plt.title('Magnitude Binary', fontsize=25)
    plt.xlabel('$\omega_x$', fontsize=20)
    plt.ylabel('$\omega_y$', fontsize=20)
    plt.xticks()
    plt.yticks()
    plt.grid(0)
    plt.savefig(folder + figfolder + name + '_thr.png')
    
    #vec = fig.add_subplot(224)
    #vec.scatter(x,y)
    #plt.title('Magnitude Binary Scatter', fontsize=25)
    #plt.xlabel('$\omega_x$', fontsize=20)
    #plt.ylabel('$\omega_y$', fontsize=20)
    #plt.ylim([0.0, size_img[0]])
    #plt.xlim([0.0, size_img[1]])
    #plt.gca().set_aspect('equal', adjustable='box')
    
        
    left  = 0.0  # the left side of the subplots of the figure
    right = 1.5    # the right side of the subplots of the figure
    bottom = 0.1   # the bottom of the subplots of the figure
    top = 1  # the top of the subplots of the figure
    wspace = 0.5   # the amount of width reserved for space between subplots,
               # expressed as a fraction of the average axis width
    hspace = 0.3   # the amount of height reserved for space between subplots,
               # expressed as a fraction of the average axis height
    # plt.subplots_adjust(left, bottom, right, top, wspace, hspace)
    
    sigma = np.sqrt(pca.explained_variance_)
    
    print('''The angle between the main component and the horizontal is 
        \u03B1 = %2.3f \n''' % (np.arctan(pca.components_[0,1]/pca.components_[0,0])\
         /np.pi/2 * 360))
    print('''The angle between the second component and the horizontal is 
        \u03B2 = %2.3f \n''' % (np.arctan(-pca.components_[1,1]/pca.components_[1,0])\
         /np.pi/2 * 360))
    # pca.explained_variance_[0] = variance along pricipal comp
    # pca.explained_variance_[1] = variance along ortogonal comp
    print("The alignment factor is: %.3g" %  (sigma[1]/sigma[0]))
    
    return (sigma[1]/sigma[0]), plt.show()

In [ ]:
# first choose the folder
folder = "Analysis_2ndpart/different_C/123TCB_CB/"
figfolder = "figures/"

#create folders if they do not exist already:
if not os.path.exists(folder + figfolder):
    os.makedirs(folder + figfolder)
    
AlignVec = np.ones(80)

#runs over all .jpg files in the folder:
for item in glob.glob(folder + "*.jpg"):
    #extracting the number of the measurement, the channel and tha name:
    #name, chanNum, measNum = hf.FindChanAndMeas(folder, item)
    
    print(item)
    
    idx = [i for i in range(len(item)) if item[i] == '/']
    name = item[idx[0]+1:np.max(idx)]
    
    idxup = item.find("sample")
    idxdown = item.find(".")
    sampleNum = item[idxup+7 : idxdown]
    if (len(sampleNum) > 2): sampleNum = sampleNum[0]
    
    alignFact,_ = find_main_component_dft(item, 3)
    
    AlignVec[int(sampleNum)-1] = alignFact
    